In [ ]:
from pathlib import Path
from utils.audio_utils import get_total_audio_duration
from utils.file_utils import save_json,load_json
from tqdm import tqdm

In [2]:
from zspeech.inference import  ZSpeech

In [4]:
client = ZSpeech.load()

INFO:zspeech.inference.client:Load VAD
Extracting onnx_model.tar.gz : 2it [00:00, 690.99it/s]
INFO:zspeech.inference.client:Load ASR
Extracting model.tar.gz : 3it [00:04,  1.56s/it]
INFO:zspeech.asr.inference._inference:intializing decoder


Total time taken for reading the FST file: 0.527969 seconds


Extracting onnx_model.tar.gz : 3it [00:01,  1.83it/s]
INFO:root:ClassifyFst restored from en_fst.far.gz
INFO:root:VerbalizeFinalFst restored from cached en_fst.far.gz
INFO:root:PostProcessingFst restored from en_fst.far.gz
INFO:zspeech.inference.client:Load SE
Extracting onnx_model.tar.gz : 2it [00:00,  6.91it/s]
Extracting onnx_model.tar.gz : 2it [00:00,  6.90it/s]
INFO:zspeech.se.inference._inference:Inference runs with 16k SE service on cpu device
Extracting onnx_model.tar.gz : 5it [00:00,  8.61it/s]
Extracting onnx_model.tar.gz : 5it [00:00,  8.56it/s]
INFO:zspeech.se.inference._inference:Inference runs with 16k SE service on cpu device


In [ ]:
audio_info="/media/dataset-harddisk/munikumar/datasets/hindi_dataset/gramvaani/unlabelled/audio_info.json"

In [ ]:
data=load_json(audio_info)

In [ ]:
ge_30_sec=0
for meta in tqdm(data):
    if float(meta["duration"])<10:
        ge_30_sec+=1
    # meta['duration']=get_total_audio_duration(meta['path'])

In [ ]:
ge_30_sec

In [ ]:
ge_30_sec

In [ ]:
audio_root=Path("/media/dataset-harddisk/munikumar/datasets/hindi_dataset/gramvaani/unlabelled/final")

In [ ]:
all_audio=list(audio_root.rglob("*.mp3"))

In [ ]:
def convert_seconds_to_hh_mm_ss(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


In [ ]:
from pydub import AudioSegment
def get_channels(audio_file):
    data=AudioSegment.from_mp3(audio_file)
    return data.channels

In [ ]:
stereo=0
for file in tqdm(all_audio):
    channels=get_channels(file)
    if channels == 2:
        stereo+=channels
stereo

In [ ]:
import torchaudio
import librosa
import torch
from pathlib import Path
def convert_stereo_to_mono(input_file):
    audio_array, sample_rate = librosa.load(input_file)
    
    if len(audio_array.shape) == 1:
        audio_array=audio_array.reshape((1,-1))
        
    waveform=torch.tensor(audio_array)
    # Check if the audio is stereo
    
    if waveform.shape[0] == 2:
        mono_waveform = torch.mean(waveform, dim=0, keepdim=True)
    else:
        mono_waveform = waveform
    
    torchaudio.save(input_file, mono_waveform, sample_rate)


In [ ]:
convert_stereo_to_mono("/media/dataset-harddisk/munikumar/utils/audio_utils/609_1242306.mp3")

In [ ]:
audio_path=Path("/media/dataset-harddisk/munikumar/datasets/hindi_dataset/gramvaani/unlabelled/final/609-1242306.mp3")

In [ ]:
audio_path.exists()

In [ ]:
from zspeech.audio.io import load_audio_using_stream_reader

In [ ]:
audio_sig,sr=load_audio_using_stream_reader(all_audio[0],to_mono=True)

In [ ]:
audio_sig.shape

In [ ]:
audio, out_sample_rate = torchaudio.load(
        audio_path, format="mp3", normalize=False, frame_offset=0,
        num_frames=-1
    )

In [ ]:
audio.shape

In [ ]:
import librosa

audio_path = "/media/dataset-harddisk/munikumar/utils/audio_utils/609_1242306.mp3"
audio_data, sample_rate = librosa.load(audio_path)


In [ ]:
len(audio_data.shape)

In [ ]:
torchaudio.save("mono.mp3",torch.tensor(audio_data.reshape(1,-1)),sample_rate)

In [ ]:
Audio(audio_data,rate=sample_rate)

In [ ]:
import pydub
import numpy as np

def read_mp3(filename, normalized=False):
    # MP3 to numpy array
    audio = pydub.AudioSegment.from_mp3(filename)
    samples = np.array(audio.get_array_of_samples())
    if audio.channels == 2:
        samples = samples.reshape((-1, 2))
    if normalized:
        return audio.frame_rate, np.float32(samples) / 2**15
    else:
        return audio.frame_rate, samples

def write_mp3(filename, sample_rate, samples, normalized=False):
    # numpy array to MP3
    channels = 2 if (samples.ndim == 2 and samples.shape[1] == 2) else 1
    if normalized:
        samples = np.int16(samples * 2 ** 15)
    else:
        samples = np.int16(samples)
    audio = pydub.AudioSegment(samples.tobytes(), frame_rate=sample_rate, sample_width=2, channels=channels)
    audio.export(filename, format="mp3", bitrate="320k")


In [ ]:
data=read_mp3("/media/dataset-harddisk/munikumar/utils/audio_utils/609_1242306.mp3",normalized=True)

In [ ]:
data[1].shape

In [ ]:
from IPython.display import Audio
Audio(data[1],rate=data[0])

In [5]:
from pathlib import Path
import os

audio_root=Path("/media/dataset-harddisk/munikumar/datasets/hindi_dataset/zvoice/split_audio")
audio_files=list(audio_root.rglob("*.wav"))
dst=Path("/media/dataset-harddisk/munikumar/datasets/hindi_dataset/zvoice/all_split_audio")
dst.mkdir(exist_ok=True,parents=True)
for file in audio_files:
    os.system(f"mv {str(file)} {str(dst)}")